In [8]:
import pandas as pd


faculty_info = pd.read_csv('../data_clean/faculty_info_clean.csv')
course_survey = pd.read_csv('../data_clean/course_survey_clean.csv')
student_comments = pd.read_csv('../data_clean/student_comments_clean.csv')


In [9]:
course_survey['rating_score'] = (
    course_survey['rating_score']
    .astype(str)
    .str.strip()
    .str.replace(',', '.')
    .replace({'': None, 'nan': None, 'NaN': None})
    .astype(float)
)

print("rating_score sau khi làm sạch (10 dòng đầu):")
print(course_survey['rating_score'].head(10))
print("\nKiểu dữ liệu:", course_survey['rating_score'].dtype)
print("Số giá trị bị thiếu (NaN):", course_survey['rating_score'].isna().sum())

rating_score sau khi làm sạch (10 dòng đầu):
0    3.50
1    3.50
2    3.50
3    3.50
4    3.50
5    4.00
6    3.50
7    4.00
8    4.00
9    4.25
Name: rating_score, dtype: float64

Kiểu dữ liệu: float64
Số giá trị bị thiếu (NaN): 0


In [10]:
df_khao_sat = course_survey.merge(faculty_info, on='faculty_id', how='left')

print("Sau khi merge thông tin giảng viên:")
print(f"Kích thước dataframe: {df_khao_sat.shape[0]} dòng, {df_khao_sat.shape[1]} cột")
print("\n5 dòng đầu:")
print(df_khao_sat[['survey_id', 'faculty_id', 'full_name', 'department', 'rating_score']].head())

Sau khi merge thông tin giảng viên:
Kích thước dataframe: 120 dòng, 8 cột

5 dòng đầu:
  survey_id faculty_id     full_name   department  rating_score
0   SRV1000      GV106  Nguyen Van A  Bộ Môn Toán           3.5
1   SRV1001      GV120    Tran Thi B    Khoa CNTT           3.5
2   SRV1002      GV111      Le Van C    Khoa CNTT           3.5
3   SRV1003      GV123      Le Van C  Bộ Môn Toán           3.5
4   SRV1004      GV105    Pham Thi D    Khoa CNTT           3.5


In [11]:
comments_agg = (
    student_comments.groupby('survey_id')
    .agg({
        'comment_text': lambda x: ' | '.join(x.dropna().astype(str)),  # Gộp comment
        'sentiment_label': 'first'  # Lấy nhãn cảm xúc đầu tiên (hoặc có thể dùng mode)
    })
    .reset_index()
)

print("Số survey_id có bình luận sau khi gộp:", comments_agg.shape[0])

# Merge vào dataframe chính
df = df_khao_sat.merge(comments_agg, on='survey_id', how='left')

# Điền giá trị mặc định cho các khảo sát không có bình luận
df['sentiment_label'] = df['sentiment_label'].fillna('Trung lập')

print("\nSau khi merge bình luận:")
print(f"Kích thước cuối cùng: {df.shape[0]} dòng, {df.shape[1]} cột")
print("\n5 dòng đầu (đầy đủ thông tin):")
print(df.head())

Số survey_id có bình luận sau khi gộp: 66

Sau khi merge bình luận:
Kích thước cuối cùng: 120 dòng, 10 cột

5 dòng đầu (đầy đủ thông tin):
  survey_id faculty_id course_code student_level  rating_score survey_term  \
0   SRV1000      GV106       DA200         Năm 2           3.5    HK2-2024   
1   SRV1001      GV120       CT102    Liên Thông           3.5   HK1- 2024   
2   SRV1002      GV111       CT101    Liên Thông           3.5   HK1- 2024   
3   SRV1003      GV123       CT101         Năm 1           3.5    HK2-2024   
4   SRV1004      GV105       DA200         Năm 1           3.5    HK1-2024   

      full_name   department                                  comment_text  \
0  Nguyen Van A  Bộ Môn Toán  Giang vien day de hieu | Bài giảng hơi nhanh   
1    Tran Thi B    Khoa CNTT                                           NaN   
2      Le Van C    Khoa CNTT                                           NaN   
3      Le Van C  Bộ Môn Toán                                           NaN   
4 

In [12]:
diem_tb_giangvien = (
    df.groupby(['faculty_id', 'full_name', 'department'], as_index=False)
    .agg(
        diem_trung_binh=('rating_score', 'mean'),
        so_khao_sat=('rating_score', 'count')
    )
    .round({'diem_trung_binh': 2})
    .sort_values('diem_trung_binh', ascending=False)
    .reset_index(drop=True)
)

print("1. Điểm trung bình theo giảng viên (top 20):")
print(diem_tb_giangvien.head(20))

#2.Điểm trung bình theo khoa/bộ môn
diem_tb_khoa = (
    df.groupby('department')['rating_score']
    .mean()
    .round(2)
    .sort_values(ascending=False)
    .reset_index(name='diem_trung_binh')
)

print("\n2. Điểm trung bình theo khoa/bộ môn:")
print(diem_tb_khoa)

1. Điểm trung bình theo giảng viên (top 20):
   faculty_id     full_name    department  diem_trung_binh  so_khao_sat
0       GV103   Hoang Van E  Khoa Kinh Tế             5.00            1
1       GV112   Hoang Van E   Bộ Môn Toán             4.50            6
2       GV100    Pham Thi D  Khoa Kinh Tế             4.33            3
3       GV113  Nguyen Van A   Bộ Môn Toán             4.25            3
4       GV101   Hoang Van E  Khoa Kinh Tế             4.15            5
5       GV111      Le Van C     Khoa CNTT             4.15           10
6       GV119      Le Van C  Khoa Kinh Tế             4.12            4
7       GV107      Le Van C  Khoa Kinh Tế             4.11            7
8       GV114    Tran Thi B   Bộ Môn Toán             4.11            7
9       GV122    Pham Thi D  Khoa Kinh Tế             4.08            6
10      GV123      Le Van C   Bộ Môn Toán             4.06            9
11      GV105    Pham Thi D     Khoa CNTT             4.04            6
12      GV121   Hoa

In [13]:
bang_cheo = (
    df.groupby(['department', 'student_level'])['rating_score']
    .mean()
    .round(2)
    .unstack(fill_value=0)
)

print("3. Điểm trung bình theo khoa và mức độ sinh viên:")
print(bang_cheo)

#4.Tỷ lệ cảm xúc theo khoa
ty_le_cam_xuc = df.groupby(['department', 'sentiment_label']).size().unstack(fill_value=0)

for label in ['Tích cực', 'Tiêu cực', 'Trung lập']:
    if label not in ty_le_cam_xuc.columns:
        ty_le_cam_xuc[label] = 0

ty_le_cam_xuc['Tong'] = ty_le_cam_xuc.sum(axis=1)
ty_le_cam_xuc['Tich_cuc_%'] = (ty_le_cam_xuc['Tích cực'] / ty_le_cam_xuc['Tong'] * 100).round(1)
ty_le_cam_xuc['Tieu_cuc_%'] = (ty_le_cam_xuc['Tiêu cực'] / ty_le_cam_xuc['Tong'] * 100).round(1)
ty_le_cam_xuc['Trung_lap_%'] = (ty_le_cam_xuc['Trung lập'] / ty_le_cam_xuc['Tong'] * 100).round(1)

print("\n4. Tỷ lệ cảm xúc theo khoa:")
print(ty_le_cam_xuc[['Tich_cuc_%', 'Tieu_cuc_%', 'Trung_lap_%']].sort_values('Tich_cuc_%', ascending=False))

3. Điểm trung bình theo khoa và mức độ sinh viên:
student_level  Liên Thông  Năm 1  Năm 2
department                             
Bộ Môn Toán          3.75   4.20   4.22
Khoa CNTT            3.88   3.91   4.14
Khoa Kinh Tế         4.00   4.22   3.94

4. Tỷ lệ cảm xúc theo khoa:
sentiment_label  Tich_cuc_%  Tieu_cuc_%  Trung_lap_%
department                                          
Khoa Kinh Tế           26.3         7.9         47.4
Khoa CNTT              23.9        10.9         54.3
Bộ Môn Toán            16.7        16.7         50.0


In [14]:
nguong = 5  #Chỉ lấy giảng viên có ít nhất 5 khảo sát

top_giangvien = diem_tb_giangvien[diem_tb_giangvien['so_khao_sat'] >= nguong].head(10)
thap_giangvien = diem_tb_giangvien[diem_tb_giangvien['so_khao_sat'] >= nguong].sort_values('diem_trung_binh').head(10)

print(f"Top 10 giảng viên ĐIỂM CAO NHẤT (≥ {nguong} khảo sát):")
print(top_giangvien[['full_name', 'department', 'diem_trung_binh', 'so_khao_sat']])

print(f"\nTop 10 giảng viên ĐIỂM THẤP NHẤT (≥ {nguong} khảo sát):")
print(thap_giangvien[['full_name', 'department', 'diem_trung_binh', 'so_khao_sat']])

#Lưu kết quả
diem_tb_giangvien.to_csv('../final_report/ket_qua_phan_tich/diem_trung_binh_giang_vien.csv', index=False, encoding='utf-8-sig')
diem_tb_khoa.to_csv('../final_report/ket_qua_phan_tich/diem_trung_binh_khoa.csv', index=False, encoding='utf-8-sig')
bang_cheo.to_csv('../final_report/ket_qua_phan_tich/bang_cheo_khoa_nam.csv', encoding='utf-8-sig')
ty_le_cam_xuc[['Tich_cuc_%', 'Tieu_cuc_%', 'Trung_lap_%']].to_csv('../final_report/ket_qua_phan_tich/ty_le_cam_xuc.csv', encoding='utf-8-sig')


Top 10 giảng viên ĐIỂM CAO NHẤT (≥ 5 khảo sát):
      full_name    department  diem_trung_binh  so_khao_sat
1   Hoang Van E   Bộ Môn Toán             4.50            6
4   Hoang Van E  Khoa Kinh Tế             4.15            5
5      Le Van C     Khoa CNTT             4.15           10
7      Le Van C  Khoa Kinh Tế             4.11            7
8    Tran Thi B   Bộ Môn Toán             4.11            7
9    Pham Thi D  Khoa Kinh Tế             4.08            6
10     Le Van C   Bộ Môn Toán             4.06            9
11   Pham Thi D     Khoa CNTT             4.04            6
15  Hoang Van E     Khoa CNTT             4.00            5
16   Tran Thi B     Khoa CNTT             4.00            5

Top 10 giảng viên ĐIỂM THẤP NHẤT (≥ 5 khảo sát):
       full_name    department  diem_trung_binh  so_khao_sat
20    Pham Thi D     Khoa CNTT             3.88            8
19    Tran Thi B  Khoa Kinh Tế             3.91            8
18    Pham Thi D     Khoa CNTT             3.92            